# Simulate curve generation defined by a parametric equation
Modified: Jul 17, 2019  
Author: Hayley Song


In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
import numpy as np
import pandas as pd
import time

In [ ]:
import holoviews as hv
from holoviews import opts, dim
from holoviews.streams import *

import panel as pn
hv.notebook_extension('bokeh')
hv.Dimension.type_formatters[np.datetime64] = '%Y-%m-%d'
pn.extension()

In [ ]:
import streamz
import streamz.dataframe

One way to represent a curve on a plane (aka. planary curve) is to specify a parametrization, which is a function from an interval $I \in R$ to $R^2$. Image taking a segment in 1D, transform it in a "smooth" way (eg. bend/stretch/rotate/translate it) and put it in a 2D space.  This means, we need to specify the followings:
- a parameter, eg. p $\in [0,1]$, which  specify where you are on the line segment
- two functions $x(p)$ and $y(p)$, which define the coordinate of the point $C(p)$ in x and y axis, respectively

## Curve Generator
- Generates a point on the curve at a given value $p$  and sends $(x(p), y(p))$ to the buffer stream, which will update the dynamic maps linked to the buffer 

In [ ]:
class CurveSimulator(param.Parameterized):
    """
    Params:
    - n_steps: Number of points on the curve, ie. number of steps in [0,1] range
    - p : parameter for curve function. It runs from 0 to 1, inclusively
    - reset : action parameter that sets the value of the p parameter to zero and clears the buffer stream
    - t_interval: time interval to pause in between sending data points while simulation is running
    """

    n_steps = param.Integer(label='Number of simulation steps', default=100)
    p = param.ObjectSelector(label='p', default=0., objects=np.linspace(0,1,num=n_steps.default))
    reset = param.Action(lambda x: x.reset_handler(), doc="Click to clear the buffer and reset p")
    t_interval = param.Number(label='t_interval', doc='Time interval between plotting two points',
                              softbounds=(0., 5.),
                              default=0.)

    
    ################################################################################
    # Constant class properties
    ################################################################################
    H,W = 500,500
    curve_opts = opts.Points(size=5,width=W, height=H, 
                             xlim=(-1,1), ylim=(-1,1),
                             color=dim('p')*256-50,
                             tools=['hover']
                            )
    xopts = opts.Points('XCoord', width=W, height=H, size=5, xlim=(0,1), 
                        padding=0.1, invert_axes=True, invert_yaxis=True)
    yopts = opts.Points('YCoord', width=W, height=H, size=5, xlim=(0,1), padding=0.1, invert_xaxis=True)
    
    
    ################################################################################
    # Parameter Dependencies
    ################################################################################    
    @param.depends('n_steps', watch=True)
    def _update_p(self):
        self.count['p'] += 1
        self.param['p'].objects = np.linspace(0,1,num=self.n_steps)
        print('updated p with new number of simulation steps: ', self.n_steps)
    
    @param.depends('p', watch=True)
    def send_point(self):
        point = pd.DataFrame([(self.p, *self.cfunc(self.p))], columns=['p','x','y'])
        self.data_src.emit(point)
        time.sleep(self.t_interval)
        
    def reset_handler(self):
        self.set_param(p=0.)
        self.dfstream.clear()

        
    ################################################################################
    # Initialization
    ################################################################################
    def __init__(self, cfunc, n_steps=100, **kwargs):
        """
        Args:
        - cfunc (function): given an input of a float p in [0,1], returns (x,y), a 
        tuple of x and y coords
        
        - n_steps (int): number of simulation steps along the range of [0,1] for 
        the parameter, p
        """
        super().__init__(**kwargs) # this is super important
        self.cfunc = cfunc 
        self.n_steps = n_steps
        
        self.example = pd.DataFrame({'p': [], 'x':[], 'y':[]})
        self.data_src = streamz.dataframe.DataFrame(example=self.example)
        self.dfstream = Buffer(self.data_src, length=min(self.n_steps, 100), index=False)
        self.set_dmap_curve()
        self.set_dmap_x()
        self.set_dmap_y()
        self.overlay = (self.dmap_curve + self.dmap_y + self.dmap_x).cols(2)
    

    def set_dmap_curve(self):
        dmap_curve = hv.DynamicMap(
            lambda data: hv.Points(data, kdims=['x','y'], group='Curve'),
            streams=[self.dfstream])#.opts(color='p')
        self.dmap_curve = dmap_curve.opts(self.curve_opts)
        
    def set_dmap_x(self):
        dmap_x = hv.DynamicMap(
            lambda data: hv.Points( data, kdims=['p','x'], group='XCoord'),
            streams=[self.dfstream]).opts(color='p')
        self.dmap_x = dmap_x.opts(self.xopts)
        
    def set_dmap_y(self):
        dmap_y = hv.DynamicMap(
            lambda data: hv.Points( data, kdims=['p','y'], group='YCoord'),
            streams=[self.dfstream]).opts(color='p')
        self.dmap_y = dmap_y.opts(self.yopts)
    
    
    ################################################################################
    # Display DynammicMaps
    ################################################################################ 
    def viewable(self):
        return self.overlay
        


### Define curve function

In [ ]:
xfunc = lambda p: np.sin(2*np.pi*p)
yfunc = lambda p: np.cos(2*np.pi*p)

In [ ]:
# Try different functions
xfunc = lambda p: np.sin(2*np.pi*p)**2
yfunc = lambda p: np.cos(2*np.pi*p)

In [ ]:
xfunc = lambda p: np.sin(2*np.pi*p)**10
yfunc = lambda p: np.cos(2*np.pi*p)

In [ ]:
# Alternating along a straight line 
xfunc = lambda p: np.sin(2*np.pi*p)**2
yfunc = lambda p: np.cos(2*np.pi*p)**2

In [ ]:
# Doesn't have to choose a periodic function
xfunc = lambda p: np.log(p)
yfunc = lambda p: p

In [ ]:
# Something happens at p=0.5
xfunc = lambda p: np.sin(2*np.pi*p**2)*p**3
yfunc = lambda p: np.sin(np.pi*p**0.5)

### Create the simulator for the curve

In [ ]:
cfunc = lambda p: (xfunc(p), yfunc(p))
c = CurveSimulator(cfunc)

### Show the simulator

In [ ]:
pn.Row(
    pn.Param(c.param, width=500, widgets={
        'p': pn.widgets.DiscretePlayer,
        'reset': pn.widgets.Button(name=c.param['reset'].label),
        't_interval': pn.widgets.FloatSlider
    }),
    pn.panel(c.viewable())
)

---
### Spirograph
Spirograph has a nice parametric equation we can try with our curve simulator. Since it has two parameters that generate different type of spirograph, let's create a subclass of CurveSimulator with these two extra parameters to play with.

![spirograph](../assets/spirograph.png)
![spiro](../assets/spiro_equation.png)

[src](https://upload.wikimedia.org/wikipedia/commons/9/90/Various_Spirograph_Designs.jpg)

In [ ]:
class SpirographSimulator(CurveSimulator):

    R = param.Number(label='R', default=1., 
                     doc="""Radius of a circle (Doesn't affect the shape)""")
    k = param.Number(label='k', default=0.5, bounds=(0., 1.), softbounds=(1e-4, 1.0),
                     doc="""Ratio between the radius of the inner circle to the outer one""")
    l = param.Number(label='l', default=0.5, bounds=(0., 1.),
                     doc="""Parameter for how far the point A is located from the inner circle""")
    
    def __init__(self, **kwargs):
        self.temp_count = defaultdict(int)
        self._update_cfunc()
        super().__init__(self.cfunc, **kwargs)
        
    @param.depends('k','R','l', watch=True)
    def _update_cfunc(self):
        self.temp_count['cfunc'] += 1
        xfunc = lambda p: self.R*( (1-self.k)*np.cos(2*np.pi*p) + self.l*self.k*np.cos( ((1-self.k)/self.k)*2*np.pi*p))
        yfunc = lambda p: self.R*( (1-self.k)*np.sin(2*np.pi*p) + self.l*self.k*np.sin( ((1-self.k)/self.k)*2*np.pi*p))
        cfunc = lambda p: (xfunc(p), yfunc(p))
        self.cfunc = cfunc
        
        

In [ ]:
spiro = SpirographSimulator()

Try different values of $k$ and $l$
eg: 
- $k, l=0.5, 0.5 $  --> circle
- $k, l = 0.1, 0.5$
- $k, l = 0.8, 0.5$

In [ ]:
# Create the widgets
pn.Row(
    pn.Param(spiro.param, width=500, widgets={
        'p': pn.widgets.DiscretePlayer,
        'reset': pn.widgets.Button,
        'k': pn.widgets.FloatSlider(name=spiro.param.k.label,
                                    value=spiro.k, 
                                    #callback_policy="mouseup" 
                                   ),
        'l': pn.widgets.FloatSlider(name=spiro.param.l.label,
                                    value=spiro.l, 
                                    #callback_policy="mouseup"
                                    ),
    }),
    pn.panel(spiro.viewable())
)